In [38]:
from __future__ import (
    division,
    print_function,
)
import sys
import os
import skimage.data
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from selective_search import selective_search
from __future__ import division
from skimage.segmentation import felzenszwalb
import skimage.io
import skimage.feature
import skimage.color
import skimage.transform
import skimage.util
import skimage.segmentation
import numpy as np
import pickle

In [39]:
# Import necessary libraries
import os
import json
import skimage.io
import selective_search
from selective_search import selective_search

def convert_numpy_to_python(data):
    """
    Convert numpy data types to native Python types for JSON serialization.
    """
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()  # Replacing np.asscalar with .item()
    return data

def save_proposals(image_path, output_path, selective_search_func):
    """
    Apply selective search on an image and save the proposals.
    """
    # Load the image
    image = skimage.io.imread(image_path)

    # Generate region proposals
    _, proposals = selective_search_func(image)

    # Filter out proposals with 0 width or height
    valid_proposals = [p for p in proposals if p['rect'][2] > 0 and p['rect'][3] > 0]

    # Convert NumPy types to native Python types for JSON serialization
    for proposal in valid_proposals:
        proposal['rect'] = convert_numpy_to_python(proposal['rect'])
        proposal['size'] = convert_numpy_to_python(proposal['size'])
        proposal['labels'] = convert_numpy_to_python(proposal['labels'])

    # Save the proposals to a file
    with open(output_path, 'w') as file:
        json.dump(valid_proposals, file)

        
def process_dataset(dataset_path, selective_search_func):
    """
    Process all images in the dataset using selective search and save the proposals.
    """
    for split in ['train', 'valid']:
        split_path = os.path.join(dataset_path, split)
        output_split_path = os.path.join(dataset_path, split + '_proposals')

        # Create directory for proposals if it doesn't exist
        if not os.path.exists(output_split_path):
            os.makedirs(output_split_path)

        for image_file in os.listdir(split_path):
            if image_file.endswith('.json') or image_file.endswith('.DS_Store'):  # Skip the annotation file
                continue

            image_path = os.path.join(split_path, image_file)
            output_path = os.path.join(output_split_path, image_file + '.json')

            save_proposals(image_path, output_path, selective_search_func)



In [40]:
dataset_path = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/'  # Replace with the actual path to your dataset
process_dataset(dataset_path, selective_search)  # Replace 'generate_segments' with your selective search function

1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extracted
4.Generating final regions
1.Segments are generated
2.Regions are extracted
3.Neighbours are extract

In [41]:
import json
import os
import numpy as np

# Load JSON data from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def calculate_iou(boxA, boxB):
    # Determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

def classify_proposals(proposals_dir, annotations, tp=0.50, tn=0.15):
    positive_samples_dict, negative_samples_dict = [], []

    for proposal_file in os.listdir(proposals_dir):
        print(proposal_file)
        if proposal_file.endswith('.json'):
            proposals = load_json(os.path.join(proposals_dir, proposal_file))
            image_base_name = os.path.splitext(proposal_file)[0]
            # Correct way to find the image_id
            image_id = next((image['id'] for image in annotations['images'] if image['file_name'].startswith(image_base_name)), None)

            if image_id is not None:
                image_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == image_id]
                for proposal in proposals:
                    #print(proposal)
                    proposal_box = [proposal['rect'][0], proposal['rect'][1], proposal['rect'][0] + proposal['rect'][2], proposal['rect'][1] + proposal['rect'][3]]
                    max_iou = 0

                    for ann in image_annotations:
                        ann_box = [ann['bbox'][0], ann['bbox'][1], ann['bbox'][0] + ann['bbox'][2], ann['bbox'][1] + ann['bbox'][3]]
                        iou = calculate_iou(proposal_box, ann_box)
                        max_iou = max(max_iou, iou)

                    if max_iou >= tp:
                        positive_samples_dict.append({'proposal': proposal, 'image_name': image_base_name})
                    elif max_iou <= tn:
                        negative_samples_dict.append({'proposal': proposal, 'image_name': image_base_name})

    return positive_samples_dict, negative_samples_dict

# Load the annotations data
annotations_data = load_json('/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train/_annotations.coco.json')

# Example usage
proposals_dir = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train_proposals' # Path to your proposals directory
positive_samples_dict, negative_samples_dict = classify_proposals(proposals_dir, annotations_data)


7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg.json
3646097131_e3e1215843_b_jpg.rf.fabadb610747501143f10d9f96086a06.jpg.json
5674044810_2d9e2243ff_b_jpg.rf.ca828dc7bd53da6ae038c6c37c68c4e5.jpg.json
4955354786_337a598e4a_b_jpg.rf.e24cd3979e50dc5a69d068cddbd865d5.jpg.json
8436015314_3a678c1005_k_jpg.rf.9112ae54c27c1d130c83f126ae04e005.jpg.json
4552737035_3a0a4105fb_b_jpg.rf.36ab5b26ef82d1fd4ec8efe34ebc4608.jpg.json
7308740338_591f27b631_k_jpg.rf.2aaf44901b1f1e0697e2b2c81b60defa.jpg.json
16435593892_2aa8118f4a_k_jpg.rf.39e9da364b7fe4f5a3955ebc56f4dc58.jpg.json
605521662_a470fef77f_b_jpg.rf.e52fb9b46ada3a323ef4dc06d8080f96.jpg.json
3927754171_9011487133_b_jpg.rf.4dba40316d5227810a9f73a6c7f2e992.jpg.json
18849792632_aad23ad513_k_jpg.rf.1d732499d34e8190aed5a7cabc46210f.jpg.json
873768102_7d931e5fa3_b_jpg.rf.181d7271311b60ed104db70a21170be8.jpg.json
14321263043_b76ef054d3_k_jpg.rf.5b0867e2fac41a1d5e21bb75ea4a3a05.jpg.json
5253122239_38b1e7f61c_b_jpg.rf.4775e501d74f8a92298

In [42]:
len(negative_samples_dict)

28321

In [43]:
len(positive_samples_dict)

611

In [7]:
negative_samples_dict[1]['proposal']['rect']

[9, 0, 3, 127]

In [8]:
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

In [13]:
def extract_features(image_folder, proposals):
    features = []
    i=0
    for element in proposals:
        i=i+1
        print(i+1)
        proposal=element['proposal']
        #print(proposal)
        x, y, w, h = proposal['rect']
        image_name=element['image_name']
        #print(image_name)
        image_path=image_folder+image_name
        #print(image_path)
        image = cv2.imread(image_path)
        #print(image.shape)
        #print(y,y+h,x,x+w)
        crop = image[y:y+h, x:x+w]
        #print(crop.shape)
        crop = cv2.resize(crop, (224, 224))
        crop = np.expand_dims(crop, axis=0)
        crop = preprocess_input(crop)

        feature = model.predict(crop)
        features.append(feature.flatten())

    return features



In [10]:
image_folder_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train/'
train_features_positive = extract_features(image_folder_train, positive_samples_dict)


{'rect': [93, 262, 37, 81], 'size': 213, 'labels': 272}
7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg
/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train/7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg
(416, 416, 3)
262 343 93 130
(81, 37, 3)
1/1 [==============================] - 0s 277ms/step
{'rect': [192, 304, 105, 111], 'size': 1858, 'labels': 9540}
7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg
/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train/7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg
(416, 416, 3)
304 415 192 297
(111, 105, 3)
1/1 [==============================] - 0s 131ms/step
{'rect': [78, 260, 36, 70], 'size': 257, 'labels': 574}
7488015492_0583857ca0_k_jpg.rf.d8c2dfaec75a67609483533b207ead08.jpg
/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer 

In [17]:
image_folder_test= '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/train/'
train_features_negative = extract_features(image_folder_train, negative_samples_dict)

2
1/1 [==============================] - 0s 113ms/step
3
1/1 [==============================] - 0s 110ms/step
4
1/1 [==============================] - 0s 111ms/step
5
1/1 [==============================] - 0s 123ms/step
6
1/1 [==============================] - 0s 114ms/step
7
1/1 [==============================] - 0s 113ms/step
8
1/1 [==============================] - 0s 117ms/step
9
1/1 [==============================] - 0s 117ms/step
10
1/1 [==============================] - 0s 123ms/step
11
1/1 [==============================] - 0s 123ms/step
12
1/1 [==============================] - 0s 110ms/step
13
1/1 [==============================] - 0s 118ms/step
14
1/1 [==============================] - 0s 149ms/step
15
1/1 [==============================] - 0s 112ms/step
16
1/1 [==============================] - 0s 111ms/step
17
1/1 [==============================] - 0s 109ms/step
18
1/1 [==============================] - 0s 111ms/step
19
1/1 [==============================] - 0s 111ms/step


In [20]:
with open('train_features_negative.pkl', 'wb') as file:
    pickle.dump(train_features_negative, file)


In [21]:
with open('train_features_positive.pkl', 'wb') as file:
    pickle.dump(train_features_positive, file)
 

## For test data

In [25]:
# Load the annotations data
annotations_data = load_json('/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/valid/_annotations.coco.json')

# Example usage
proposals_dir_test= '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/valid_proposals' # Path to your proposals directory
positive_samples_dict_test, negative_samples_dict_test = classify_proposals(proposals_dir_test, annotations_data)

12288043903_fe1ea17a4e_k_jpg.rf.f5f2bcdf7876adf9e8d942ba03f470f9.jpg.json
15331928994_d5b82eb368_k_jpg.rf.1af049d1afe14ee46ef90338701636dd.jpg.json
126700562_8e27720147_b_jpg.rf.5245aa428c2def8f97691dcecc8eead2.jpg.json
34020010494_e5cb88e1c4_k_jpg.rf.07f0734f98a42af1ce2e13e55d91b343.jpg.json
2311771643_f46392fcc0_b_jpg.rf.74f0e88be59a94b6b5c3520f2fe49ad5.jpg.json
9330497995_4cf0438cb6_k_jpg.rf.4fca4a8ac43c2bb5cc5736be74dcdbda.jpg.json
15717689633_5f7f78c28e_k_jpg.rf.e06a4042bf85d09a23c9f6ca56f497e8.jpg.json
12037308314_e16fb3a0f7_k_jpg.rf.ffa282b12dea77d5dfa8336e7d47a185.jpg.json
4543126482_92254ef046_b_jpg.rf.b31cd7e849cf6d4cefb5206bc6c90407.jpg.json
699765866_abaad7274d_b_jpg.rf.79959bca6ca40ea0c8028d41fc7bac02.jpg.json
321888854_3723b6f10b_b_jpg.rf.37edc48dc2054732265be859fc1ff3a5.jpg.json
6483318883_21facf57cd_b_jpg.rf.e801140899f2dbe80112d1d5a34bae33.jpg.json


In [28]:
len(positive_samples_dict_test)

116

In [30]:
len(negative_samples_dict_test)

8242

In [31]:
image_folder_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/valid/'
test_features_positive = extract_features(image_folder_test, positive_samples_dict_test)

2
1/1 [==============================] - 0s 284ms/step
3
1/1 [==============================] - 0s 132ms/step
4
1/1 [==============================] - 0s 133ms/step
5
1/1 [==============================] - 0s 135ms/step
6
1/1 [==============================] - 0s 135ms/step
7
1/1 [==============================] - 0s 139ms/step
8
1/1 [==============================] - 0s 142ms/step
9
1/1 [==============================] - 0s 137ms/step
10
1/1 [==============================] - 0s 138ms/step
11
1/1 [==============================] - 0s 136ms/step
12
1/1 [==============================] - 0s 137ms/step
13
1/1 [==============================] - 0s 140ms/step
14
1/1 [==============================] - 0s 139ms/step
15
1/1 [==============================] - 0s 132ms/step
16
1/1 [==============================] - 0s 135ms/step
17
1/1 [==============================] - 0s 130ms/step
18
1/1 [==============================] - 0s 133ms/step
19
1/1 [==============================] - 0s 154ms/step


In [32]:
image_folder_test= '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 5/exercise-5/data/balloon_dataset/valid/'
test_features_negative = extract_features(image_folder_test, negative_samples_dict_test)

2
1/1 [==============================] - 0s 134ms/step
3
1/1 [==============================] - 0s 139ms/step
4
1/1 [==============================] - 0s 139ms/step
5
1/1 [==============================] - 0s 169ms/step
6
1/1 [==============================] - 0s 137ms/step
7
1/1 [==============================] - 0s 136ms/step
8
1/1 [==============================] - 0s 139ms/step
9
1/1 [==============================] - 0s 139ms/step
10
1/1 [==============================] - 0s 150ms/step
11
1/1 [==============================] - 0s 165ms/step
12
1/1 [==============================] - 0s 147ms/step
13
1/1 [==============================] - 0s 139ms/step
14
1/1 [==============================] - 0s 140ms/step
15
1/1 [==============================] - 0s 172ms/step
16
1/1 [==============================] - 0s 153ms/step
17
1/1 [==============================] - 0s 142ms/step
18
1/1 [==============================] - 0s 138ms/step
19
1/1 [==============================] - 0s 143ms/step


In [33]:
with open('test_features_negative.pkl', 'wb') as file:
    pickle.dump(test_features_negative, file)

with open('test_features_positive.pkl', 'wb') as file:
    pickle.dump(test_features_positive, file)


## Training Model 

In [36]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming train_features_positive and train_features_negative are lists of feature vectors
X_train = np.concatenate((train_features_positive, train_features_negative))
y_train = np.array([1] * len(train_features_positive) + [0] * len(train_features_negative))

# Initialize the SVM classifier
clf = SVC(C=1.0, kernel='rbf', gamma='scale',class_weight='balanced')

# Train the classifier
clf.fit(X_train, y_train)

# Evaluate the classifier
X_test = np.concatenate((test_features_positive, test_features_negative))
y_test = np.array([1] * len(test_features_positive) + [0] * len(test_features_negative))

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9899497487437185
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8242
           1       1.00      0.28      0.43       116

    accuracy                           0.99      8358
   macro avg       0.99      0.64      0.71      8358
weighted avg       0.99      0.99      0.99      8358



In [37]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming train_features_positive and train_features_negative are lists of feature vectors
X_train = np.concatenate((train_features_positive, train_features_negative))
y_train = np.array([1] * len(train_features_positive) + [0] * len(train_features_negative))

# Initialize the SVM classifier
clf = SVC(C=1.0, kernel='rbf', gamma='scale',class_weight='balanced')

# Train the classifier
clf.fit(X_train, y_train)

# Evaluate the classifier
X_test = np.concatenate((test_features_positive, test_features_negative))
y_test = np.array([1] * len(test_features_positive) + [0] * len(test_features_negative))

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.990667623833453
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8242
           1       1.00      0.33      0.49       116

    accuracy                           0.99      8358
   macro avg       1.00      0.66      0.74      8358
weighted avg       0.99      0.99      0.99      8358



In [46]:
from imblearn.under_sampling import RandomUnderSampler

# Combine positive and negative training features
X_train = np.concatenate((train_features_positive, train_features_negative))
y_train = np.array([1] * len(train_features_positive) + [0] * len(train_features_negative))

# Apply Random Under Sampling
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

In [52]:
# Initialize the SVM classifier
clf = SVC(C=1.0, kernel='rbf', gamma='scale')

# Train the classifier
clf.fit(X_res, y_res)

# Evaluate the classifier
X_test = np.concatenate((test_features_positive, test_features_negative))
y_test = np.array([1] * len(test_features_positive) + [0] * len(test_features_negative))

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9930605407992342
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      8242
           1       0.70      0.88      0.78       116

    accuracy                           0.99      8358
   macro avg       0.85      0.94      0.89      8358
weighted avg       0.99      0.99      0.99      8358



In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto'],  # Kernel coefficient
    'kernel': ['rbf', 'linear']  # Specifies the kernel type to be used in the algorithm
}

# Initialize the SVM classifier
svc = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', verbose=2)

# Fit GridSearchCV
grid_search.fit(X_res, y_res)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Evaluate the best model
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)

print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Classification Report on Test Set:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.7s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.6s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   1.7s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.5s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.5s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.6s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.6s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.5s
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time=   2.8s
[CV] END ......................C=0.1, gamma=auto

In [54]:
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 0.9230846436935429


In [61]:
import joblib
joblib.dump(clf, 'svm_classifier.pkl')


['svm_classifier.pkl']